In [210]:
import numpy as np
rand = np.random.default_rng(42)

In [211]:
class Layer:
    def __init__(self):
        pass 

    def __call__(self,x,train=False):
        pass 

    def backward(self,out_grad,learning_rate):
        pass 

In [212]:
class Linear(Layer):
    def __init__(self,fan_in,fan_out,seed=42):
        self.fan_in = fan_in 
        self.fan_out = fan_out

        limit = np.sqrt(6 / (fan_in + fan_out))
        self.weights = np.random.uniform(-limit, limit, size=(fan_out, fan_in))
        # self.weights = np.random.randn(fan_out,fan_in)
        self.bias = np.ones((fan_out,1))
        # print(self.weights.shape)
        # print(self.grad.shape)
    def __call__(self,x,train=False):
        self.input = x.reshape(self.fan_in,-1)
        return np.dot(self.weights,self.input) + self.bias
    

    def __repr__(self):
        pass 

    def backward(self, out_grad, learning_rate ):
        wgrad = np.dot(out_grad, self.input.T) 
        bgrad = out_grad 
        inputgrad = np.dot(self.weights.T, out_grad)

        self.weights -= learning_rate * wgrad 
        self.bias -= learning_rate * bgrad 

        # print(f"{self.wgrad}")
        # print(f"{self.bgrad}")

        return inputgrad 

    def reset_grad(self):
        self.wgrad = np.zeros((self.fan_in,self.fan_out))
        self.bgrad = np.zeros((1,self.fan_out)) 


In [213]:
class Softmax(Layer):
    def __call__(self, input, train=False):
        self.input = input
        tmp = input - max(input)  
        tmp = np.exp(tmp)
        self.output = tmp / np.sum(tmp)
        return self.output 
    def backward(self, out_grad, learning_rate):
        n = np.size(self.output) 
        return np.dot((np.identity(n)-self.output.T) * self.output, out_grad)  

In [214]:
class Activation(Layer):
    def __init__(self,activation,activation_prime):
        self.activation = activation 
        self.activation_grad = activation_prime

    def __call__(self, input, train=False):
        self.input = input 
        return self.activation(self.input) 
    
    def backward(self, out_grad, learning_rate):
        return np.multiply(out_grad, self.activation_grad(self.input))

In [215]:
class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)
        def tanh_grad(x):
            return 1-np.tanh(x)**2 
        super().__init__(tanh,tanh_grad)

In [216]:
class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1.0 / (1.0 + np.exp(-x))
        
        def sigmoid_grad(x):
            return sigmoid(x) * (1.0 - sigmoid(x)) 
        super().__init__(sigmoid,sigmoid_grad) 
            

In [217]:
class ReLU(Activation):
    def __init__(self):
        def relu(x):
            return np.maximum(0,x)
        def relu_grad(x):
            return np.where(x > 0, 1, np.where(x < 0, 0, 0.5))
        super().__init__(relu,relu_grad)

In [218]:
class Dropout(Layer):
    def __init__(self, dropout_rate):
        self.dropout_rate = dropout_rate
        self.mask = None

    def __call__(self, x, train=False):
        if train:
            self.mask = (np.random.rand(*x.shape) < (1 - self.dropout_rate)) / (1 - self.dropout_rate)
            # print(self.mask)
            return x * self.mask
        else:
            return x

    def backward(self, grad, learning_rate):
        return grad * self.mask if self.mask is not None else grad

In [219]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_grad(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)

def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_grad(y_true, y_pred): # wrt y_pred
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

def cross_entropy(y_true,y_pred):
    return np.mean(-y_true*np.log(y_pred)) 

def cross_entropy_grad(y_true,y_pred):
    return np.mean(-y_true/y_pred) 

In [220]:
class NN:
    def __init__(self, *layers):
        self.layers : Layer = [] 
        for layer in layers:
            self.layers += [layer]
        
    def __call__(self,input,train=False):
        for layer in self.layers:
            input = layer(input,train=train)
        return input 
    def backward(self, ):
        pass 
    
    def save(self,filename):
        pass 
    def load(self,filename):
        pass
    
    def train_minibatch(self,):
        pass 
    
    def train_batch(self, loss, loss_grad, X, y, epochs = 1000, learning_rate = 0.001, verbose=True):
        for epoch in range(epochs):
            error = 0 
            for x, y in zip(X,y):
                output = self.__call__(x,train=True)
                error += loss(y,output)

                grad = loss_grad(y,output)

                for layer in reversed(self.layers):
                    grad = layer.backward(grad, learning_rate)
            error /= len(X)
            if verbose:
                print(f"{epoch=}, {error=}")

    def eval(self, loss, X,y):
        error = 0 
        for x,y in zip(X,y):
            output = self.__call__(x)
            error += loss(y,output)
        error /= len(X)
        return error 
    


In [221]:
lin = Linear(10,5)
relu = ReLU() 
sigmoid = Sigmoid()
softmax = Softmax()

model = NN(lin, sigmoid, relu, softmax) 

x = rand.random(10).reshape(-1,1)

print(x)
model(x)

[[0.77395605]
 [0.43887844]
 [0.85859792]
 [0.69736803]
 [0.09417735]
 [0.97562235]
 [0.7611397 ]
 [0.78606431]
 [0.12811363]
 [0.45038594]]


array([[0.19428353],
       [0.24264922],
       [0.22592517],
       [0.14571562],
       [0.19142646]])

In [222]:
model = NN(Linear(32,5), ReLU(), Linear(5,10), Softmax()) 

x = np.random.randn(1,32)

print(x)
result = model(x) 
print(result, np.sum(result))


[[ 0.12145991  0.13581939  1.07936877 -0.02456193 -1.4789741  -0.1496096
   0.45248678  2.18440467 -0.94909598  0.96902005 -0.27567245  0.29392967
  -0.20771191  0.92493014 -0.05235155  0.67805099  2.01729155 -0.9919856
  -0.04054623  0.01893221 -0.33492212 -0.43392444 -1.36067623 -0.72886045
   0.04237201  0.09486662 -0.8063684   0.28603057 -1.7058329  -0.92626847
  -1.04479309 -0.89894544]]
[[0.10411696]
 [0.4448591 ]
 [0.1441088 ]
 [0.01760836]
 [0.1479912 ]
 [0.00629213]
 [0.02052927]
 [0.04333672]
 [0.06622795]
 [0.00492951]] 1.0


Import Dataset

In [223]:
from torchvision import datasets, transforms 

train_validation_dataset = datasets.EMNIST(root='./data', 
                                           split='letters',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True,
                                        )


independent_test_dataset = datasets.EMNIST(
                             root='./data',
                             split='letters',
                             train=False,
                             transform=transforms.ToTensor(),
                             )


In [224]:
train_validation_dataset.target_transform
train_validation_dataset.targets
len(train_validation_dataset.data)
len(train_validation_dataset.targets)

124800

In [225]:
type(train_validation_dataset)
n_classes = len(train_validation_dataset.classes)
n_datapoints = len(train_validation_dataset.targets)

X_train = np.array(train_validation_dataset.data.reshape(-1,28*28))
# y_train = np.zeros((len(train_validation_dataset.targets),len(train_validation_dataset.classes)))

y_train = np.array([[1 if temp == i else 0 for i in range(n_classes)] for temp in train_validation_dataset.targets])
y_train[3]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [226]:
train_validation_dataset.targets[1]
y_train[1]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [227]:
X_test = np.array(independent_test_dataset.data.reshape(-1,28*28))
y_test = np.array([[1 if temp == i else 0 for i in range(n_classes)] for temp in independent_test_dataset.targets])

In [228]:
y_test[:3]

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0]])

In [229]:
import pandas as pd 

df = pd.DataFrame(train_validation_dataset.data.reshape(-1,28*28))

In [230]:
df[:5]

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [231]:
model = NN(Linear(28, 26), Dropout(0.5))

In [232]:
x = np.random.randn(28,1)
x
drop = Dropout(0.8)

In [233]:
drop(x, train=True)

array([[-0.7771404 ],
       [-0.        ],
       [ 0.        ],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [-9.33086507],
       [-0.        ],
       [-0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [-0.        ],
       [ 0.        ],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [ 0.        ],
       [ 0.        ],
       [-0.        ],
       [ 0.        ],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [-4.97264438],
       [ 0.        ],
       [-0.        ]])

In [234]:
model = NN(Linear(28*28,1024), ReLU(), Dropout(0.2), Linear(1024,26), ReLU(), Dropout(0.3), Sigmoid())

In [235]:
model.train_batch(cross_entropy, cross_entropy_grad, X_train, y_train, 3)